In [2]:
import numpy as np
import pandas as pd
import scipy.io.wavfile as wav
from scipy.fftpack import dct
import os

In [3]:
THRESHOLD = 0.05
ALPHA = 0.97
FRAME_SIZE = 0.025
FRAME_STRIDE = 0.01
SAMPLE_RATE = 16000
NUM_FILTERS = 26
MIN_FREQ = 0
N_FFT = 512
WINDOW_SIZE = 32768
NUM_CEPS = 13

In [4]:
def remove_silence(signal, threshold = THRESHOLD):
    signal[np.abs(signal) < threshold] = 0
    return signal

In [5]:
def pre_emphasis(signal, alpha = ALPHA):
    emphasized_signal = np.append(signal[0], signal[1:] - alpha * signal[:-1])
    return emphasized_signal


In [6]:
def framing(signal, frame_size = FRAME_SIZE, frame_stride = FRAME_STRIDE, sample_rate = SAMPLE_RATE):
    frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate
    signal_length = len(signal)
    frame_length = int(round(frame_length))
    frame_step = int(round(frame_step))
    num_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step))

    padded_signal_length = num_frames * frame_step + frame_length
    z = np.zeros((padded_signal_length - signal_length))
    pad_signal = np.append(signal, z) 

    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    frames = pad_signal[indices.astype(np.int32, copy = False)]
    return frames

In [7]:
def apply_window(frames, window_func = np.hamming):
    frames *= window_func(frames.shape[1])
    return frames

In [8]:
def create_mel_filterbank(sample_rate, num_filters = NUM_FILTERS, min_freq = MIN_FREQ, max_freq = None, n_fft = N_FFT):
    if max_freq is None:
        max_freq = sample_rate // 2
    #  The formula to convert frequency in Hertz to Mel is given by: Mel=2595×log_10(1+𝑓/700)
    mel_min = 2595 * np.log10(1 + min_freq / 700)
    mel_max = 2595 * np.log10(1 + max_freq / 700)
    mel_points = np.linspace(mel_min, mel_max, num_filters + 2)
    hz_points = 700 * (10**(mel_points / 2595) - 1)
    bin_points = np.floor((n_fft + 1) * hz_points / sample_rate).astype(int)
    
    filterbank = np.zeros((num_filters, n_fft // 2 + 1))
    for i in range(1, num_filters + 1):
        filterbank[i - 1, bin_points[i - 1]:bin_points[i]] = (bin_points[i] - bin_points[i - 1]) / (hz_points[i] - hz_points[i - 1])
        filterbank[i - 1, bin_points[i]:bin_points[i + 1]] = (bin_points[i + 1] - bin_points[i]) / (hz_points[i + 1] - hz_points[i])
    return filterbank

In [9]:
def compute_mfcc(signal, sample_rate = SAMPLE_RATE, num_ceps = NUM_CEPS):
    frames = framing(signal, sample_rate = sample_rate)
    frames *= WINDOW_SIZE
    frames = apply_window(frames)
    magnitude_spectrum = np.abs(np.fft.rfft(frames, n = N_FFT))
    mel_filterbank = create_mel_filterbank(sample_rate, num_filters = NUM_FILTERS, n_fft = N_FFT)
    mel_spectrum = np.dot(magnitude_spectrum, mel_filterbank.T)
    log_mel_spectrum = np.log(mel_spectrum + 1e-10) 
    mfcc = dct(log_mel_spectrum, type = 2, axis = 1, norm = 'ortho')[:, 1 : (num_ceps + 1)]  
    return mfcc

In [10]:
hisb_60_warsh_transcript_df = pd.read_excel('warsh_transcript/warshData_v2-1.xlsx', skiprows = 1)
hisb_60_warsh_transcript_df.reset_index(drop = True, inplace = True)
hisb_60_warsh_transcript_df = hisb_60_warsh_transcript_df.iloc[:-1]
Al_Fatihah = hisb_60_warsh_transcript_df[hisb_60_warsh_transcript_df['sura_no'] == 1]
hisb_60_warsh_transcript_df = hisb_60_warsh_transcript_df[(hisb_60_warsh_transcript_df['sura_no'] >= 87) & (hisb_60_warsh_transcript_df['sura_no'] <= 114)]

hisb_60_and_Al_Fatihah_warsh_transcript_df = pd.concat([Al_Fatihah, hisb_60_warsh_transcript_df])

In [11]:
hisb_60_and_Al_Fatihah_warsh_transcript_df = hisb_60_and_Al_Fatihah_warsh_transcript_df[['sura_no',
                                            'sura_name_en',
                                            'sura_name_ar',
                                            'aya_no',
                                            'aya_text']]
hisb_60_and_Al_Fatihah_warsh_transcript_df.index = range(298)
hisb_60_and_Al_Fatihah_warsh_transcript_df

sura_no sura_name_en sura_name_ar  aya_no  \
0        1.0   Al-Fātiḥah   الفَاتِحة      1.0   
1        1.0   Al-Fātiḥah   الفَاتِحة      2.0   
2        1.0   Al-Fātiḥah   الفَاتِحة      3.0   
3        1.0   Al-Fātiḥah   الفَاتِحة      4.0   
4        1.0   Al-Fātiḥah   الفَاتِحة      5.0   
..       ...          ...          ...     ...   
293    114.0       An-Nās     النَّاس      2.0   
294    114.0       An-Nās     النَّاس      3.0   
295    114.0       An-Nās     النَّاس      4.0   
296    114.0       An-Nās     النَّاس      5.0   
297    114.0       An-Nās     النَّاس      6.0   

                                        aya_text  
0      اِ۬لْحَمْدُ لِلهِ رَبِّ اِ۬لْعَٰلَمِينَ ١  
1                  اَ۬لرَّحْمَٰنِ اِ۬لرَّحِيمِ ٢  
2                    مَلِكِ يَوْمِ اِ۬لدِّينِۖ ٣  
3    إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُۖ ٤  
4     اُ۪هْدِنَا اَ۬لصِّرَٰطَ اَ۬لْمُسْتَقِيمَ ٥  
..                                           ...  
293                          مَلِكِ اِ۬لنَّاسِ ٢  
294                         إِلَٰهِ اِ۬لنَّاسِ ٣  
295     مِن شَرِّ اِ۬لْوَسْوَاسِ اِ۬لْخَنَّاسِ ٤  
296  اِ۬لذِے يُوَسْوِسُ فِے صُدُورِ اِ۬لنَّاسِ ٥  
297              مِنَ اَ۬لْجِنَّةِ وَالنَّاسِۖ ٦  

[298 rows x 5 columns]

In [12]:
directories = ["safa"]

In [13]:
safa_wav = []
wavs = [safa_wav]
for i in range(len(directories)) :
    for subdir_name in os.listdir(directories[i]):
        subdir_path = os.path.join(directories[i], subdir_name)
        if os.path.isdir(subdir_path):
            for filename in os.listdir(subdir_path):
                file_path = os.path.join(subdir_path, filename)
                file_path = file_path.replace("\\", "/")
                wavs[i].append(file_path)

In [14]:
safa_recitor = {
    'recitor_en': "safa",
    'recitor_ar': "صفاء",
    'recitor_reciting_aya_wav_path': wavs[0],
}

safa_recitor_df = pd.DataFrame(safa_recitor, index = None)
safa_recitor_df

recitor_en recitor_ar recitor_reciting_aya_wav_path
0         safa       صفاء              safa/001/001.wav
1         safa       صفاء              safa/001/002.wav
2         safa       صفاء              safa/001/003.wav
3         safa       صفاء              safa/001/004.wav
4         safa       صفاء              safa/001/005.wav
..         ...        ...                           ...
293       safa       صفاء              safa/114/002.wav
294       safa       صفاء              safa/114/003.wav
295       safa       صفاء              safa/114/004.wav
296       safa       صفاء              safa/114/005.wav
297       safa       صفاء              safa/114/006.wav

[298 rows x 3 columns]

In [15]:
safa_hisb_60_and_Al_fatihah_audio_with_transcript = pd.concat([hisb_60_and_Al_Fatihah_warsh_transcript_df,
                                                                    safa_recitor_df], axis=1)
safa_hisb_60_and_Al_fatihah_audio_with_transcript

sura_no sura_name_en sura_name_ar  aya_no  \
0        1.0   Al-Fātiḥah   الفَاتِحة      1.0   
1        1.0   Al-Fātiḥah   الفَاتِحة      2.0   
2        1.0   Al-Fātiḥah   الفَاتِحة      3.0   
3        1.0   Al-Fātiḥah   الفَاتِحة      4.0   
4        1.0   Al-Fātiḥah   الفَاتِحة      5.0   
..       ...          ...          ...     ...   
293    114.0       An-Nās     النَّاس      2.0   
294    114.0       An-Nās     النَّاس      3.0   
295    114.0       An-Nās     النَّاس      4.0   
296    114.0       An-Nās     النَّاس      5.0   
297    114.0       An-Nās     النَّاس      6.0   

                                        aya_text recitor_en recitor_ar  \
0      اِ۬لْحَمْدُ لِلهِ رَبِّ اِ۬لْعَٰلَمِينَ ١       safa       صفاء   
1                  اَ۬لرَّحْمَٰنِ اِ۬لرَّحِيمِ ٢       safa       صفاء   
2                    مَلِكِ يَوْمِ اِ۬لدِّينِۖ ٣       safa       صفاء   
3    إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُۖ ٤       safa       صفاء   
4     اُ۪هْدِنَا اَ۬لصِّرَٰطَ اَ۬لْمُسْتَقِيمَ ٥       safa       صفاء   
..                                           ...        ...        ...   
293                          مَلِكِ اِ۬لنَّاسِ ٢       safa       صفاء   
294                         إِلَٰهِ اِ۬لنَّاسِ ٣       safa       صفاء   
295     مِن شَرِّ اِ۬لْوَسْوَاسِ اِ۬لْخَنَّاسِ ٤       safa       صفاء   
296  اِ۬لذِے يُوَسْوِسُ فِے صُدُورِ اِ۬لنَّاسِ ٥       safa       صفاء   
297              مِنَ اَ۬لْجِنَّةِ وَالنَّاسِۖ ٦       safa       صفاء   

    recitor_reciting_aya_wav_path  
0                safa/001/001.wav  
1                safa/001/002.wav  
2                safa/001/003.wav  
3                safa/001/004.wav  
4                safa/001/005.wav  
..                            ...  
293              safa/114/002.wav  
294              safa/114/003.wav  
295              safa/114/004.wav  
296              safa/114/005.wav  
297              safa/114/006.wav  

[298 rows x 8 columns]

In [16]:
data = []
for index, row in safa_hisb_60_and_Al_fatihah_audio_with_transcript.iterrows():
    # Load audio file
    sample_rate, signal = wav.read(row['recitor_reciting_aya_wav_path'])
    # Remove silence
    signal = remove_silence(signal)
    # Apply pre-emphasis
    signal = pre_emphasis(signal)
    # Compute MFCCs
    mfcc = compute_mfcc(signal, sample_rate = sample_rate)
    data.append(mfcc.tolist())

mfcc_df = pd.DataFrame({'mfcc': data})
mfcc_df

mfcc
0    [[-6.863160586364426, 1.8010427338849118, 1.40...
1    [[-6.626498898017648, 2.0574360709601773, 1.40...
2    [[-6.051009192374478, 1.8341372173989545, 0.28...
3    [[-7.3398516909280795, 0.3754188496212656, -0....
4    [[-5.941233984302933, 1.1045472631121722, -0.2...
..                                                 ...
293  [[-7.515438694849998, -0.3372102289950032, 0.1...
294  [[-6.226748728199061, 1.3069454108447056, 0.48...
295  [[-6.487375206439677, 1.027782709882356, 0.413...
296  [[-5.325851012318113, 0.2791443963545781, -0.6...
297  [[-5.47758853978773, 0.9124991080038554, 0.602...

[298 rows x 1 columns]

In [17]:
safa_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC = pd.concat([safa_hisb_60_and_Al_fatihah_audio_with_transcript,
                                                                    mfcc_df], axis = 1)
safa_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC

sura_no sura_name_en sura_name_ar  aya_no  \
0        1.0   Al-Fātiḥah   الفَاتِحة      1.0   
1        1.0   Al-Fātiḥah   الفَاتِحة      2.0   
2        1.0   Al-Fātiḥah   الفَاتِحة      3.0   
3        1.0   Al-Fātiḥah   الفَاتِحة      4.0   
4        1.0   Al-Fātiḥah   الفَاتِحة      5.0   
..       ...          ...          ...     ...   
293    114.0       An-Nās     النَّاس      2.0   
294    114.0       An-Nās     النَّاس      3.0   
295    114.0       An-Nās     النَّاس      4.0   
296    114.0       An-Nās     النَّاس      5.0   
297    114.0       An-Nās     النَّاس      6.0   

                                        aya_text recitor_en recitor_ar  \
0      اِ۬لْحَمْدُ لِلهِ رَبِّ اِ۬لْعَٰلَمِينَ ١       safa       صفاء   
1                  اَ۬لرَّحْمَٰنِ اِ۬لرَّحِيمِ ٢       safa       صفاء   
2                    مَلِكِ يَوْمِ اِ۬لدِّينِۖ ٣       safa       صفاء   
3    إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُۖ ٤       safa       صفاء   
4     اُ۪هْدِنَا اَ۬لصِّرَٰطَ اَ۬لْمُسْتَقِيمَ ٥       safa       صفاء   
..                                           ...        ...        ...   
293                          مَلِكِ اِ۬لنَّاسِ ٢       safa       صفاء   
294                         إِلَٰهِ اِ۬لنَّاسِ ٣       safa       صفاء   
295     مِن شَرِّ اِ۬لْوَسْوَاسِ اِ۬لْخَنَّاسِ ٤       safa       صفاء   
296  اِ۬لذِے يُوَسْوِسُ فِے صُدُورِ اِ۬لنَّاسِ ٥       safa       صفاء   
297              مِنَ اَ۬لْجِنَّةِ وَالنَّاسِۖ ٦       safa       صفاء   

    recitor_reciting_aya_wav_path  \
0                safa/001/001.wav   
1                safa/001/002.wav   
2                safa/001/003.wav   
3                safa/001/004.wav   
4                safa/001/005.wav   
..                            ...   
293              safa/114/002.wav   
294              safa/114/003.wav   
295              safa/114/004.wav   
296              safa/114/005.wav   
297              safa/114/006.wav   

                                                  mfcc  
0    [[-6.863160586364426, 1.8010427338849118, 1.40...  
1    [[-6.626498898017648, 2.0574360709601773, 1.40...  
2    [[-6.051009192374478, 1.8341372173989545, 0.28...  
3    [[-7.3398516909280795, 0.3754188496212656, -0....  
4    [[-5.941233984302933, 1.1045472631121722, -0.2...  
..                                                 ...  
293  [[-7.515438694849998, -0.3372102289950032, 0.1...  
294  [[-6.226748728199061, 1.3069454108447056, 0.48...  
295  [[-6.487375206439677, 1.027782709882356, 0.413...  
296  [[-5.325851012318113, 0.2791443963545781, -0.6...  
297  [[-5.47758853978773, 0.9124991080038554, 0.602...  

[298 rows x 9 columns]

In [18]:
ahkam_indexing = pd.read_excel('ahkam_indexing/Ahkam in ayat hisb 60 + 001_v2.xlsx')

In [19]:
columns_to_keep = ["madd_6_Lazim", "madd_246", "madd_6", "madd_2",
                    "Ikhfaa", "Idgham", "tafkhim", "qalqala", "imala"]
new_df = ahkam_indexing[columns_to_keep]

In [20]:
new_df

madd_6_Lazim madd_246 madd_6 madd_2                    Ikhfaa    Idgham  \
0           [-1]     [-1]   [36]   [31]                      [-1]      [-1]   
1           [-1]     [24]   [-1]   [11]                      [-1]   [3, 18]   
2           [-1]     [21]   [-1]   [-1]                      [-1]      [17]   
3           [-1]     [37]   [-1]   [-1]                      [-1]      [-1]   
4           [-1]     [36]   [-1]   [19]                      [-1]      [13]   
..           ...      ...    ...    ...                       ...       ...   
293         [-1]     [14]   [-1]   [-1]              [11, 12, 13]      [10]   
294         [-1]     [15]   [-1]    [4]              [12, 13, 14]      [11]   
295         [-1]     [35]   [-1]   [-1]           [2, 32, 33, 34]      [-1]   
296         [-1]     [38]   [-1]   [-1]              [35, 36, 37]      [34]   
297         [-1]     [25]   [-1]   [-1]  [12, 13, 14, 22, 23, 24]  [20, 21]   

                                  tafkhim qalqala imala  
0                                [18, 19]    [-1]  [-1]  
1                   [4, 5, 6, 19, 20, 21]    [-1]  [-1]  
2                                    [-1]    [-1]  [-1]  
3                                    [-1]    [-1]  [-1]  
4    [14, 15, 16, 17, 18, 20, 21, 34, 35]    [-1]  [-1]  
..                                    ...     ...   ...  
293                                  [-1]    [-1]  [-1]  
294                                  [-1]    [-1]  [-1]  
295                              [30, 31]    [-1]  [-1]  
296                              [23, 24]    [-1]  [-1]  
297                                  [-1]    [-1]  [-1]  

[298 rows x 9 columns]

In [21]:
special_characters = ['١', '٢', '٣', '٤', '٥', '٦', '٧', '٨', '٩', '٠', 'ۖ', '۞', '۩']
for char in special_characters:
    safa_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC['aya_text'] = safa_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC['aya_text'].str.replace(char, '')
safa_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC['aya_text'] = safa_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC['aya_text'].str.strip()

In [22]:
safa_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC

sura_no sura_name_en sura_name_ar  aya_no  \
0        1.0   Al-Fātiḥah   الفَاتِحة      1.0   
1        1.0   Al-Fātiḥah   الفَاتِحة      2.0   
2        1.0   Al-Fātiḥah   الفَاتِحة      3.0   
3        1.0   Al-Fātiḥah   الفَاتِحة      4.0   
4        1.0   Al-Fātiḥah   الفَاتِحة      5.0   
..       ...          ...          ...     ...   
293    114.0       An-Nās     النَّاس      2.0   
294    114.0       An-Nās     النَّاس      3.0   
295    114.0       An-Nās     النَّاس      4.0   
296    114.0       An-Nās     النَّاس      5.0   
297    114.0       An-Nās     النَّاس      6.0   

                                      aya_text recitor_en recitor_ar  \
0      اِ۬لْحَمْدُ لِلهِ رَبِّ اِ۬لْعَٰلَمِينَ       safa       صفاء   
1                  اَ۬لرَّحْمَٰنِ اِ۬لرَّحِيمِ       safa       صفاء   
2                     مَلِكِ يَوْمِ اِ۬لدِّينِ       safa       صفاء   
3     إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ       safa       صفاء   
4     اُ۪هْدِنَا اَ۬لصِّرَٰطَ اَ۬لْمُسْتَقِيمَ       safa       صفاء   
..                                         ...        ...        ...   
293                          مَلِكِ اِ۬لنَّاسِ       safa       صفاء   
294                         إِلَٰهِ اِ۬لنَّاسِ       safa       صفاء   
295     مِن شَرِّ اِ۬لْوَسْوَاسِ اِ۬لْخَنَّاسِ       safa       صفاء   
296  اِ۬لذِے يُوَسْوِسُ فِے صُدُورِ اِ۬لنَّاسِ       safa       صفاء   
297               مِنَ اَ۬لْجِنَّةِ وَالنَّاسِ       safa       صفاء   

    recitor_reciting_aya_wav_path  \
0                safa/001/001.wav   
1                safa/001/002.wav   
2                safa/001/003.wav   
3                safa/001/004.wav   
4                safa/001/005.wav   
..                            ...   
293              safa/114/002.wav   
294              safa/114/003.wav   
295              safa/114/004.wav   
296              safa/114/005.wav   
297              safa/114/006.wav   

                                                  mfcc  
0    [[-6.863160586364426, 1.8010427338849118, 1.40...  
1    [[-6.626498898017648, 2.0574360709601773, 1.40...  
2    [[-6.051009192374478, 1.8341372173989545, 0.28...  
3    [[-7.3398516909280795, 0.3754188496212656, -0....  
4    [[-5.941233984302933, 1.1045472631121722, -0.2...  
..                                                 ...  
293  [[-7.515438694849998, -0.3372102289950032, 0.1...  
294  [[-6.226748728199061, 1.3069454108447056, 0.48...  
295  [[-6.487375206439677, 1.027782709882356, 0.413...  
296  [[-5.325851012318113, 0.2791443963545781, -0.6...  
297  [[-5.47758853978773, 0.9124991080038554, 0.602...  

[298 rows x 9 columns]

In [23]:
safa_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC_and_ahkam_indexing = pd.concat([safa_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC, new_df], axis = 1)


In [24]:
safa_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC_and_ahkam_indexing

sura_no sura_name_en sura_name_ar  aya_no  \
0        1.0   Al-Fātiḥah   الفَاتِحة      1.0   
1        1.0   Al-Fātiḥah   الفَاتِحة      2.0   
2        1.0   Al-Fātiḥah   الفَاتِحة      3.0   
3        1.0   Al-Fātiḥah   الفَاتِحة      4.0   
4        1.0   Al-Fātiḥah   الفَاتِحة      5.0   
..       ...          ...          ...     ...   
293    114.0       An-Nās     النَّاس      2.0   
294    114.0       An-Nās     النَّاس      3.0   
295    114.0       An-Nās     النَّاس      4.0   
296    114.0       An-Nās     النَّاس      5.0   
297    114.0       An-Nās     النَّاس      6.0   

                                      aya_text recitor_en recitor_ar  \
0      اِ۬لْحَمْدُ لِلهِ رَبِّ اِ۬لْعَٰلَمِينَ       safa       صفاء   
1                  اَ۬لرَّحْمَٰنِ اِ۬لرَّحِيمِ       safa       صفاء   
2                     مَلِكِ يَوْمِ اِ۬لدِّينِ       safa       صفاء   
3     إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ       safa       صفاء   
4     اُ۪هْدِنَا اَ۬لصِّرَٰطَ اَ۬لْمُسْتَقِيمَ       safa       صفاء   
..                                         ...        ...        ...   
293                          مَلِكِ اِ۬لنَّاسِ       safa       صفاء   
294                         إِلَٰهِ اِ۬لنَّاسِ       safa       صفاء   
295     مِن شَرِّ اِ۬لْوَسْوَاسِ اِ۬لْخَنَّاسِ       safa       صفاء   
296  اِ۬لذِے يُوَسْوِسُ فِے صُدُورِ اِ۬لنَّاسِ       safa       صفاء   
297               مِنَ اَ۬لْجِنَّةِ وَالنَّاسِ       safa       صفاء   

    recitor_reciting_aya_wav_path  \
0                safa/001/001.wav   
1                safa/001/002.wav   
2                safa/001/003.wav   
3                safa/001/004.wav   
4                safa/001/005.wav   
..                            ...   
293              safa/114/002.wav   
294              safa/114/003.wav   
295              safa/114/004.wav   
296              safa/114/005.wav   
297              safa/114/006.wav   

                                                  mfcc madd_6_Lazim madd_246  \
0    [[-6.863160586364426, 1.8010427338849118, 1.40...         [-1]     [-1]   
1    [[-6.626498898017648, 2.0574360709601773, 1.40...         [-1]     [24]   
2    [[-6.051009192374478, 1.8341372173989545, 0.28...         [-1]     [21]   
3    [[-7.3398516909280795, 0.3754188496212656, -0....         [-1]     [37]   
4    [[-5.941233984302933, 1.1045472631121722, -0.2...         [-1]     [36]   
..                                                 ...          ...      ...   
293  [[-7.515438694849998, -0.3372102289950032, 0.1...         [-1]     [14]   
294  [[-6.226748728199061, 1.3069454108447056, 0.48...         [-1]     [15]   
295  [[-6.487375206439677, 1.027782709882356, 0.413...         [-1]     [35]   
296  [[-5.325851012318113, 0.2791443963545781, -0.6...         [-1]     [38]   
297  [[-5.47758853978773, 0.9124991080038554, 0.602...         [-1]     [25]   

    madd_6 madd_2                    Ikhfaa    Idgham  \
0     [36]   [31]                      [-1]      [-1]   
1     [-1]   [11]                      [-1]   [3, 18]   
2     [-1]   [-1]                      [-1]      [17]   
3     [-1]   [-1]                      [-1]      [-1]   
4     [-1]   [19]                      [-1]      [13]   
..     ...    ...                       ...       ...   
293   [-1]   [-1]              [11, 12, 13]      [10]   
294   [-1]    [4]              [12, 13, 14]      [11]   
295   [-1]   [-1]           [2, 32, 33, 34]      [-1]   
296   [-1]   [-1]              [35, 36, 37]      [34]   
297   [-1]   [-1]  [12, 13, 14, 22, 23, 24]  [20, 21]   

                                  tafkhim qalqala imala  
0                                [18, 19]    [-1]  [-1]  
1                   [4, 5, 6, 19, 20, 21]    [-1]  [-1]  
2                                    [-1]    [-1]  [-1]  
3                                    [-1]    [-1]  [-1]  
4    [14, 15, 16, 17, 18, 20, 21, 34, 35]    [-1]  [-1]  
..                                    ...     ...   ...  
293                                  [-1] 

In [25]:
safa_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC_and_ahkam_indexing.to_csv('ahkam_indexing/safa_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC_and_ahkam_indexing.csv', index = False)